In [3]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import preprocess_input

In [4]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        assert os.path.exists(X_path), f"X_path {X_path} does not exist."
        assert os.path.exists(y_path), f"y_path {y_path} does not exist."
        
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1612, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1612,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  # Use appropriate preprocessing
        y_batch = np.eye(2)[self.y[batch_indices]]  # Ensure correct one-hot encoding
        return X_batch, y_batch

# Updated Callbacks
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0

# Updated Model Building Function
def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  

    sequence_input = Input(shape=(seq_length, height, width, channels))
    
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128, return_sequences=True)(flattened_features)  # Output is 3D
    lstm_out = Dropout(0.4)(lstm_out)
    lstm_out = LSTM(128)(lstm_out) 
    lstm_out = Dropout(0.3)(lstm_out)


    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.3)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

indices = np.arange(1612)
train_indices, val_indices = train_test_split(indices, test_size=0.3, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_1612.dat', 
                          r'E:\PosePerfect\Dataset Creation\y_final_1612.dat', 
                          train_indices, batch_size=8)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_1612.dat', 
                        r'E:\PosePerfect\Dataset Creation\y_final_1612.dat', 
                        val_indices, batch_size=8)

checkpoint_dir = './EfficientNetB0_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    return lr * 0.9 if epoch >= 2 else lr

lr_callback = LearningRateScheduler(scheduler)
step_timer_callback = StepTimerCallback()
batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

# Model Training
cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 7, 7, 1280) │     4,049,571 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 62720)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 15, 128)        │    32,178,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,368,229 (138.73 MB)

 Trainable params: 32,318,658 (123.29 MB)

 Non-trainable params: 4,049,571 (15.45 MB)


--- Starting Epoch 1 ---
Epoch 1/10
Step 1/141 - Loss: 0.6914, Accuracy: 0.2500, Time: 147.65 seconds
  1/141 ━━━━━━━━━━━━━━━━━━━━ 6:04:40 156s/step - accuracy: 0.2500 - loss: 0.6914Step 2/141 - Loss: 0.6985, Accuracy: 0.4375, Time: 3.88 seconds
  2/141 ━━━━━━━━━━━━━━━━━━━━ 9:41 4s/step - accuracy: 0.3438 - loss: 0.6950     Step 3/141 - Loss: 0.6608, Accuracy: 0.5000, Time: 3.55 seconds
  3/141 ━━━━━━━━━━━━━━━━━━━━ 8:53 4s/step - accuracy: 0.3958 - loss: 0.6836Step 4/141 - Loss: 0.6703, Accuracy: 0.5625, Time: 3.47 seconds
  4/141 ━━━━━━━━━━━━━━━━━━━━ 8:31 4s/step - accuracy: 0.4375 - loss: 0.6803Step 5/141 - Loss: 0.7004, Accuracy: 0.5500, Time: 3.49 seconds
  5/141 ━━━━━━━━━━━━━━━━━━━━ 8:19 4s/step - accuracy: 0.4600 - loss: 0.6843Step 6/141 - Loss: 0.6906, Accuracy: 0.5417, Time: 3.41 seconds
  6/141 ━━━━━━━━━━━━━━━━━━━━ 8:09 4s/step - accuracy: 0.4736 - loss: 0.6853Step 7/141 - Loss: 0.7026, Accuracy: 0.5536, Time: 3.39 seconds
  7/141 ━━━━━━━━━━━━━━━━━━━━ 8:00 4s/step - accuracy:

KeyboardInterrupt: 